# UMR

## 导入模块

In [1]:
import numpy as np
import pandas as pd
import feather
from matplotlib import pyplot as plt

## 读入数据

### 日线数据

In [2]:
price_1d = feather.read_dataframe('../data/StockPriceK1d_20241231.feather')
price_1d = price_1d[(price_1d['date'] >= '2023-01-01') & (price_1d['date'] <= '2023-12-31')]
price_1d = price_1d.set_index('date')
price_1d

,date,issue,preclose,open,high,low,close,numTrades,volume,value,adj,ret,is_limit_buy,is_limit_sell
5095,2023-01-03,000001,13.16,13.20,13.85,13.05,13.77,153969.0,219412794.0,2.971547e+09,113.9362,0.046353,0.0,0.0
5096,2023-01-04,000001,13.77,13.71,14.42,13.63,14.32,145553.0,218968253.0,3.110729e+09,113.9362,0.039942,0.0,0.0
5097,2023-01-05,000001,14.32,14.40,14.74,14.37,14.48,110434.0,166542518.0,2.417272e+09,113.9362,0.011173,0.0,0.0
5098,2023-01-06,000001,14.48,14.50,14.72,14.48,14.62,88527.0,119574471.0,1.747915e+09,113.9362,0.009669,0.0,0.0
5099,2023-01-09,000001,14.62,14.75,14.88,14.52,14.80,85023.0,105765911.0,1.561368e+09,113.9362,0.012312,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14923890,2023-12-25,689009,31.06,30.85,31.20,30.06,30.08,5695.0,3591121.0,1.096494e+08,1.0000,-0.031552,0.0,0.0
14923891,2023-12-26,689009,30.08,30.14,30.25,26.00,27.85,13831.0,9042296.0,2.519455e+08,1.0000,-0.074136,0.0,0.0
14923892,2023-12-27,689009,27.85,27.90,28.89,27.18,28.89,13530.0,5488847.0,1.551564e+08,1.0000,0.037343,0.0,0.0
14923893,2023-12-28,689009,28.89,28.58,29.85,28.44,29.20,9638.0,5027247.0,1.472011e+08,1.0000,0.010730,0.0,0.0


### 分钟线测试数据

In [3]:
price_1m = feather.read_dataframe('../data/StockPriceK1m/2023/StockPriceK1m_20230103.feather')
price_1m['date'] = pd.to_datetime(price_1m['date'], format='ISO8601')
price_1m

,time,issue,date,open,high,low,close,volume,value,num_trades
0,93000,000001,2023-01-03,13.20,13.23,13.06,13.16,7251637.0,95493022.0,2244.0
1,93100,000001,2023-01-03,13.16,13.18,13.11,13.14,1130100.0,14838560.0,956.0
2,93200,000001,2023-01-03,13.14,13.17,13.11,13.15,1218694.0,16014132.0,797.0
3,93300,000001,2023-01-03,13.17,13.19,13.16,13.19,1272057.0,16763417.0,628.0
4,93400,000001,2023-01-03,13.20,13.20,13.16,13.18,1090287.0,14366090.0,715.0
...,...,...,...,...,...,...,...,...,...,...
1177195,145500,689009,2023-01-03,30.67,30.70,30.59,30.59,25519.0,783215.0,22.0
1177196,145600,689009,2023-01-03,30.59,30.69,30.59,30.69,7382.0,226170.0,23.0
1177197,145700,689009,2023-01-03,30.69,30.69,30.69,30.69,1099.0,33728.0,2.0
1177198,145800,689009,2023-01-03,30.69,30.69,30.69,30.69,0.0,0.0,0.0


## 计算风险指标

### 真实波动

In [4]:
%%timeit
tr = pd.merge(
    price_1m[['date', 'time', 'issue', 'high', 'low', 'close']],
    price_1d.loc['2023-01-03', ['date', 'issue', 'preclose']],
    on=['date', 'issue'],
    how='left'
)
tr['close_prev'] = (
    tr \
        .groupby('issue')['close'] \
        .shift(1).fillna(tr['preclose']) \
)
tr['tr1'] = tr['high'] - tr['low']
tr['tr2'] = np.abs(tr['high'] - tr['close_prev'])
tr['tr3'] = np.abs(tr['low'] - tr['close_prev'])
tr['r'] = tr[['tr1', 'tr2', 'tr3']].max(axis=1) / tr['close_prev']
tr = tr[['date', 'time', 'issue', 'r']]
tr

695 ms ± 3.03 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
